In [1]:
import sys
import os

In [2]:
from tensorflow import keras;
import tensorflow as tf

2024-01-30 18:09:10.967843: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-01-30 18:09:10.987668: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-30 18:09:10.987687: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-30 18:09:10.988173: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-30 18:09:10.991443: I tensorflow/core/platform/cpu_feature_guar

In [3]:
import torch

In [5]:
src_pt_model_path = "../retrained_models_after_pruned/retrained_model_20240124123209_acc_95.45455169677734_795th_epoch.pt"
keras_model_path = "../keras_models/retrain_pruned_cp_weights.h5"

In [24]:
src_model = torch.load(src_pt_model_path)['weight']
print(src_model)

OrderedDict([('sfeb.0.weight', tensor([[[[ 0.1428,  0.2197,  0.0902,  0.7726, -0.2171, -0.3066, -0.3669,
           -0.1093, -0.0491]]],


        [[[ 0.1764,  0.0352,  0.0069, -0.2478, -0.0381, -0.2278,  0.1628,
            0.0215, -0.4161]]],


        [[[-0.4900, -0.5628,  0.0257,  0.5765,  0.0897,  0.2853, -0.2619,
            0.0167,  0.0854]]],


        [[[ 0.1156,  0.1198, -0.2422,  0.3798, -0.8617,  0.7016,  0.1073,
           -0.9706,  0.5258]]],


        [[[-0.1486, -0.2087, -0.0362,  0.2223,  0.0827,  0.0716, -0.2701,
            0.0952,  0.5798]]]], device='cuda:0')), ('sfeb.1.weight', tensor([0.7654, 0.5080, 0.6433, 0.5627, 0.5179], device='cuda:0')), ('sfeb.1.bias', tensor([0.0662, 0.0404, 0.5229, 0.6069, 0.0809], device='cuda:0')), ('sfeb.1.running_mean', tensor([ 0.0003, -0.0009, -0.0004, -0.0002,  0.0007], device='cuda:0')), ('sfeb.1.running_var', tensor([0.0353, 0.0101, 0.0472, 0.0070, 0.0162], device='cuda:0')), ('sfeb.1.num_batches_tracked', tensor(11550, device='

In [7]:
keras_model = keras.models.load_model(keras_model_path)

2024-01-30 18:13:54.301583: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-01-30 18:13:54.302185: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-01-30 18:13:54.302249: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [8]:
keras_model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 1, 30225, 1)]     0         
                                                                 
 conv2d_12 (Conv2D)          (None, 1, 15109, 5)       45        
                                                                 
 batch_normalization_12 (Ba  (None, 1, 15109, 5)       20        
 tchNormalization)                                               
                                                                 
 re_lu_12 (ReLU)             (None, 1, 15109, 5)       0         
                                                                 
 conv2d_13 (Conv2D)          (None, 1, 7553, 32)       800       
                                                                 
 batch_normalization_13 (Ba  (None, 1, 7553, 32)       128       
 tchNormalization)                                         

In [23]:
mapping = zip(keras_model.layers, src_model)
print(type(mapping))
print(src_model['sfeb.0.weight'])
for keras_layer in keras_model.layers:
    # Set the weights of the corresponding layers in the Keras model
    if keras_layer.name == 'input_2':
        pass
        # keras_layer.set_weights(src_model)
        
   

<class 'zip'>
tensor([[[[ 0.1428,  0.2197,  0.0902,  0.7726, -0.2171, -0.3066, -0.3669,
           -0.1093, -0.0491]]],


        [[[ 0.1764,  0.0352,  0.0069, -0.2478, -0.0381, -0.2278,  0.1628,
            0.0215, -0.4161]]],


        [[[-0.4900, -0.5628,  0.0257,  0.5765,  0.0897,  0.2853, -0.2619,
            0.0167,  0.0854]]],


        [[[ 0.1156,  0.1198, -0.2422,  0.3798, -0.8617,  0.7016,  0.1073,
           -0.9706,  0.5258]]],


        [[[-0.1486, -0.2087, -0.0362,  0.2223,  0.0827,  0.0716, -0.2701,
            0.0952,  0.5798]]]], device='cuda:0')
